In [1]:
import pandas as pd

In [ ]:
!pip install selenium
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [4]:
def scrape_facebook_hashtag(hashtag, num_posts):
    # Set up Chrome WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)

    # Navigate to the Facebook hashtag page
    url = f'https://www.facebook.com/hashtag/{hashtag}'
    driver.get(url)

    # Wait for a few seconds to let the page load
    time.sleep(10)

    # Scroll to load more posts until num_posts is reached or no more posts to load
    while len(driver.find_elements(By.CSS_SELECTOR, 'div[dir="auto"][style="text-align: start;"]')) < num_posts:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(2)

    # Extract text from the specified <div> tag
    div_texts = driver.find_elements(By.CSS_SELECTOR, 'div[dir="auto"][style="text-align: start;"]')
    text_list = [div.text for div in div_texts[:num_posts]]

    # Close the browser
    driver.quit()

    return text_list


In [12]:
# hashtags=[
#     'wildfire',
#     'earthquake',
#     'flood',
#     'hurricane',
#     'typhoon',
#     'tornado',
#     'tsunami',
#     'drought',
#     'famine',
#     'epidemic',
#     'pandemic',
#     'outbreak',
#     'accident',
#     'disaster'
# ]

In [5]:
hashtags=[
    'earthquake',
    'flood',
    'hurricane',
    'typhoon',
    'tornado',
    'tsunami',
    'accident',
    'disaster'
]

In [7]:
texts = scrape_facebook_hashtag('disaster', num_posts=3)
print(texts)

['OMG \n   #fypシ゚ #lifestyle #reels #disaster #messi #viral #funny #AmaZing #cr7 #happylife #fypシ゚viral #challenge #storms #OMG', '28,000 rubber ducks were released in the ocean. \n This is what happened! #rubberducky #rubberducks #ocean #disaster #oceanfacts #insane', 'The hot oil did not in fact fix my \n. I removed it and glued on another one \n. #nails #manicure #beauty #disaster #burn #cooking #mistake #reaction #worklife #travel #recipe #storytime #accident']


In [8]:
df_all = pd.DataFrame(columns=['Hashtag', 'Text'])

# Loop through each hashtag and scrape text
for hashtag in hashtags:
    texts = scrape_facebook_hashtag(hashtag, num_posts=3)
    df_temp = pd.DataFrame({'Hashtag': [hashtag] * len(texts), 'Text': texts})
    df_all = pd.concat([df_all, df_temp], ignore_index=True)


In [9]:
df_all.head()

,Hashtag,Text
0,earthquake,Did vou feel that earthquake?
1,earthquake,An estimated magnitude 4.2 earthquake was repo...
2,earthquake,Source: Sources: Android Earthquake Alerts Sys...
3,flood,"Flooding in Catarman, Northern Samar \n #Catar..."
4,flood,"#sponsored I'll confess that in the past, I've..."


In [11]:
df_all.shape

(24, 2)

In [12]:
df_all.to_csv('fb_scraped.csv',index=False)